#### Dataclasses

In [1]:
from importlib import reload
import coordinates
reload(coordinates)
from coordinates import Coordinate
moscow = Coordinate(55.76, 37.62)
moscow   #return object

location = Coordinate(55.76, 37.62)
location == moscow
(location.lat, location.long) == (moscow.lat, moscow.long)

moscow, location  #__repr__ is no very useful

#to access those locations we have to type in

moscow.lat, moscow.long

(55.76, 37.62)

#### How to make it useful

In [20]:
#one can be using the namedtuple 

from collections import namedtuple

Coordinate = namedtuple('Coordinates', 'lat lon')
#                       ^^^^^^^^^^^^^------>Name of the named tuple not the previously defined class
issubclass(Coordinate, tuple)
Coordinate
moscow = Coordinate(lat=55.756, lon=37.617)
moscow

#More meaning ful __repr__ object

Coordinates(lat=55.756, lon=37.617)

In [2]:
#new named tuple that allows to add a datatype annotation to each field

import typing
Coordinate = typing.NamedTuple('Coordinate', [('lat', float),('long',float)])
issubclass(Coordinate, tuple)
typing.get_type_hints(Coordinate)   #get the type hints

{'lat': float, 'long': float}

In [3]:
#the typing.NamedTuple can also be constructed this way

Coordinate = typing.NamedTuple('Coordinate', lat=float, long=float)

In [4]:
typing.get_type_hints(Coordinate)

{'lat': float, 'long': float}

In [5]:
#from the python cookbook
import time
from functools import wraps

def timethis(func):
    '''Decorator that reports the execution time.'''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [6]:
A = lambda x: x*7

def countdown(n):
    while n>0:
            n-=1

timethis(countdown)(99)


countdown 1.3828277587890625e-05


In [7]:
##Decorator is a function that accepts a function as input and returns a new function as ooutput

print(r'foo\\bar\nbaz')

foo\\bar\nbaz


In [8]:
#using dataclasses

from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f},{abs(self.lon):.1f} {we}'

In [9]:
import inspect


I = Coordinate(12,12)
I.__annotations__ #not recommended

#instead
inspect.get_annotations(Coordinate)


{'lat': float, 'lon': float}

#### Mutable instances

In [10]:
##Defining a named tuple and using it

from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))

In [11]:
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [12]:
#defined attributes can also be accessed
tokyo.population, tokyo.coordinates

(36.933, (35.689722, 139.691667))

In [13]:
City._fields

('name', 'country', 'population', 'coordinates')

In [14]:
#named tuple manipulation

City._fields
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)   #also a way to make a named tuple

#Convert to dictionary
delhi._asdict()


import json
del_json = json.dumps(delhi._asdict())
print(del_json)

{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}


In [15]:
#Type checkers are meaningless in runtime 
#they are used by the ide for typecheck and inform the user about the same

import typing
class Coordinate(typing.NamedTuple):
    lat: float
    lon: float

Coordinate(None, None)
#shows no warnings or erros
#but checking the linter will reflect the type eroor

Coordinate(lat=None, lon=None)

In [16]:
#Class annotations and their uses
class DemoPlainClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [17]:
DemoPlainClass().__annotations__
#DemoPlainClass().a  #But a is not accessible as it is not an attribute
#while b is accessible

DemoPlainClass().b

1.1

In [18]:
#That was just a plain demo class
#now lets build the same using typing.NamedTuple

class DemoPlainClass(typing.NamedTuple):
    a: int    #becomes an annotation as well as instance attribute
    b: float =  1.1
    c = 'spam'


DemoPlainClass(a='ham')   #asks for a as no default value is set
#but calling the class only return a nd b not c

DemoPlainClass.__annotations__ #c is not an annotation
DemoPlainClass.a, DemoPlainClass.b, DemoPlainClass.c

#here a nd b are called descriptors



(_tuplegetter(0, 'Alias for field number 0'),
 _tuplegetter(1, 'Alias for field number 1'),
 'spam')

In [19]:
DemoPlainClass.c = 'ham'

#the annotated attributes of the demoplainclass is immutable

#where as c in immutable

In [48]:
DemoPlainClass.a = 'ham'
#But nothing happens

I  =  DemoPlainClass('ham')
#I.a = 'spam'   #attribute error / is readonly

In [45]:
DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [49]:
## Same as before but decorated with dataclass

from dataclasses import dataclass

@dataclass
class DemoDataClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [62]:
I = DemoDataClass(33)
I.a, I.b


I.a = 43   #mutable attributes

I  #Like before it doesnt get shown because it is a class attribute and not an instance attribiute


I.d = 'damn'
I

DemoDataClass(a=43, b=1.1)

In [2]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)  #to set the default value

    #guests: list = [] wont work


#also insed teh list the type can be specified

@dataclass
class ClubMember:
    name: str
    guests: list[str] = field(default_factory=list)   #str is typechecker and wont throw errors if numbers passed instead

In [3]:
ClubMember('python', guests=[1,2,3])

ClubMember(name='python', guests=[1, 2, 3])

In [4]:
##The attributes passed in creation is automatically shown when __repr__
#one can alos choose what to show in repr and what not


@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)
    athlete: bool = field(default=False, repr=False)


ClubMember('python')   #athlete doesnt get shown in repr

ClubMember(name='python', guests=[])

### Post-init Processing

In [5]:
#creating a HackerClubMember


from dataclasses import dataclass

@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()       #empty set
    handle: str = ''          #empty str
    
    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already exists'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)

In [ ]:
#Initialization of variables that are not fields

from dataclasses import InitVar


@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[DatabaseType] = None

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database.lookup('j')

In [20]:
#@dataclass Example: Dubln Core Resource Record

from dataclasses import dataclass, field
from typing import Optional
from enum import Enum, auto
from datetime import date

class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    """Media resource description"""
    identifier: str
    title: str = ''  #empty string object
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)


In [21]:
description = 'Improving the design of existing code'
book = Resource('978-fasjdhfjk', 'Refactoring, 2nd ed', ['Martin','Kent'],date(2018,11,19), ResourceType.BOOK, description, 'EN')

In [24]:
def match_asian_countries():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia', country=country):
                results.append(country)
    return results


match_asian_countries()

NameError: name 'cities' is not defined

In [16]:
#The repr generated by the dataclass is alright but we can make it more meaningful

from dataclasses import dataclass, field
from typing import Optional
from enum import Enum, auto
from datetime import date

class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    """Media resource description"""
    identifier: str
    title: str = ''  #empty string object
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''

    def __repr__(self) -> str:
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']
        for f in field(cls):
            value = getattr(self, f.name)
            res.append(f'{indent}{f.name} = {value!r},')
        
        res.append(')')
        return '\n'.join(res)

#### Pattern matching with class instances

In [19]:
#Three types of class patterns are there namely simple keyword and positional


#### Simple Class Patterns

In [20]:
#match x:
 #   case float():            #DANGER !!!
  #      do_something_with(x)

### Keyword Class Patterns

In [21]:
import typing

class City(typing.NamedTuple):
    continent: str
    name: str
    country: str

cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'São Paulo', 'BR'),
]

In [22]:
def city_match_asia():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia'):
                results.append(city)
    return results

In [23]:
city_match_asia()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [34]:
#To return the cities only

In [32]:
def city_match():
    results = []
    for city in cities:
        match city:
            case City(name=aa):
                results.append(aa)
    return results

In [33]:
city_match()

['Tokyo', 'Delhi', 'Mexico City', 'New York', 'São Paulo']

In [39]:
#To match the countries
def country_match():
    res = []
    for cntry in cities:
        match cntry:
            case City(country=cc):
                res.append(cc)
    return res

In [40]:
country_match()

['JP', 'IN', 'MX', 'US', 'BR']